# Summary analysis: Kinetic parameters 6 - contrast

## Jump Distance on Knot trajectory

### speed = 0.7 px / frame



In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import os
import fnmatch

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns

propsFn='props.csv'
props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
expFile=props['allExpFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\FigS5Contrast\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'

if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

os.chdir('..\\')

import functions.matrixUtilities_joh as mu
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
pfp.paper()
inToCm=2.54


In [ ]:
info=pd.read_csv(expFile, sep=',')#pd.read_csv(expFile,quotechar='"', sep=',', converters={'bdGroup':ast.literal_eval})
info=info[info.stimulusProtocol=='6']
info

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
    
for index,row in info.iterrows():
    startDir=RawDataDir+row.path+'\\'
    #startDir='D:\\data\\b\\2017\\'+row.path+'\\'
    #if not os.path.isdir(startDir):
    #    startDir='E:\\b\\2017\\'+row.path+'\\'
        
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
info['txtPath']=posPath
info['pairList']=PLPath

info['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 0 # flag to compute swim bout frequency (takes time, default: 1)
info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir

info['expTime']=expTime

csvFile=os.path.join(ProcessingDir,'FigS4.csv')
info.to_csv(csvFile,encoding='utf-8')
info

In [ ]:
def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp.animalIndex=tmp.animalIndex+((i)*15)
    df=pd.concat([df,tmp])
    i+=1
df['episode']=[x.strip().replace('_','') for x in df['episode']]

print('df shape',df.shape)
d=df.time
r=datetime(2017,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]
t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df

## Habituation or Fatigue within 20 hours?

Plot shoaling index during closed loop skype episodes over time.

In [ ]:
sns.tsplot(data=df, time="inDishTime",value="si",unit="animalIndex",condition="episode",estimator=np.nanmean,interpolate=False,err_style="ci_bars");
plt.xlim([0,8*60])

In [ ]:
idx=(df['inDishTime']<350) & (df['inDishTime']>45) #& (df.age==22)
episodeNames=df['episode'].unique()
dfDR=df[idx]
tmp=dfDR.groupby(['episode','animalIndex'],sort=True)[['si']]
xax=np.arange(episodeNames.shape[0])

err=tmp.std().unstack().values.T
tmp=tmp.mean().unstack()
tmp['xax']=xax
fig, axes = plt.subplots(figsize=(10, 7))





axes=tmp.plot(x='xax',kind='line',marker='o',yerr=err,
                                  linestyle=':',ax=axes,legend=False)

dfSkype=df[idx]
dfSkype=dfSkype[dfSkype['episode']==episodeNames[-1]]
mn=dfSkype.si.mean()
er=dfSkype.si.std()




axes.set_ylabel('attraction index')
plt.xlim([0,xax.max()+xax.max()*0.1])
axes.axhline(0,ls=':',color='k')
axes.set_title('Individual Pair Disc Size Tuning, Night vs. Day, n=8 pairs');


In [ ]:
tmp=dfDR.groupby(['episode','animalIndex'],sort=True)[['si']]
tmp2=tmp.mean().reset_index()
tmp2.head()

In [ ]:
plotOrder=[1,6,4,2,7,5,3,0]
epiNames=tmp2.episode.unique()
fig, ax = plt.subplots(figsize=(10,10))

plt.axhline(0)
sns.boxplot(x=tmp2['episode'],y=tmp2['si'],order=epiNames[plotOrder])
sns.swarmplot(x=tmp2['episode'],y=tmp2['si'],linewidth=1,edgecolor='gray',order=epiNames[plotOrder])
plt.ylim([-.1,.6])

In [ ]:
groupMeans=tmp2.groupby(['episode']).si.mean()[plotOrder].values
groupStd=tmp2.groupby(['episode']).si.std()[plotOrder].values
groupMeans=np.stack([np.hstack([groupMeans[:4],np.nan]),np.hstack([np.nan,groupMeans[4:]])]).T
groupStd=np.stack([np.hstack([groupStd[:4],np.nan]),np.hstack([np.nan,groupStd[4:]])]).T

In [ ]:
inToCm=2.54
fig, axes = plt.subplots(nrows=2, ncols=1, sharex='col', sharey=False,
                               gridspec_kw={'height_ratios': [5, 2]},
                               figsize=(7/inToCm,7/inToCm))

gra=np.linspace(0,1,5).repeat(5).reshape((-1,5)).T

dfx=pd.DataFrame(groupMeans,columns=['black dot','white dot'])
axes[0]=dfx.plot(kind='line',marker='o',ls='',
                 ax=axes[0],color=['k','w'],ms=3,
                 markeredgecolor='k',markeredgewidth=1,
                legend=False)

xax=np.repeat(np.arange(5),2)
axes[0].errorbar(xax.ravel(),groupMeans.ravel(),groupStd.ravel(),ls='',color='k',capthick=1)

plt.xlim([-.5,4.5])
plt.ylim([0,0.6])
axes[0].axhline(0,linestyle=':',color='k')
axes[1].imshow(np.linspace(0,1,5).reshape((1,5)),interpolation='none',cmap='binary')#,aspect='auto')

xl=[0,1,1,2,2,3,3,4]
yl=[-.2,-.2,.2,-.2,.2,-.2,.2,.2][::-1]
dc=[1,1,0,1,0,1,0,0][::-1]

for i in range(len(xl)):
    x=xl[i]
    y=yl[i]
    d=dc[i]
    c=plt.Circle((x,y),.1,facecolor=[d,d,d],edgecolor='k',linewidth=0)
    axes[1].add_artist(c)
    
axes[1].grid(b=False)
axes[1].set_ylim([-.4,.4])
axes[1].set_yticks([-.2,.2])
axes[0].set_yticks([0,.2,.4])
axes[1].set_yticklabels(['Black','White'][::-1])
axes[1].set_xticks(range(5))
axes[1].set_xticklabels(['']);
axes[1].set_xlabel('Background contrast',labelpad=-5)
axes[0].set_ylabel('Attraction')
axes[0].set_ylim([-.05,.45]);
axes[0].tick_params(axis='x', which='both',length=0)
axes[1].tick_params(axis='x', which='both',length=0)
plt.subplots_adjust(wspace=0, hspace=0)
sns.despine()
figPath=outputDir+'\\S5_Contrast.svg'
plt.savefig(figPath,bbox_inches='tight')

from shutil import copy2

def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts



for i,row in info.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\ROI*.csv')[0])
    copyList.append(glob.glob(head+'\\PositionTxt*.txt')[0])
    copyList.append(glob.glob(head+'\\PL*.txt')[0])
    copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    
    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
